In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import splitfolders
from sklearn.model_selection import train_test_split

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (128, 128)

In [3]:
dataset_dir = './defungi'
output_dir = './Train_test_split'

#Split into train, test, validation
splitfolders.ratio(dataset_dir, output=output_dir, seed=42, ratio=(.6, .2, .2), group_prefix=None)

In [4]:
train_data_dir = './Train_test_split/train'
test_data_dir = './Train_test_split/test'
val_data_dir = './Train_test_split/val'

In [5]:
train_data = tf.keras.utils.image_dataset_from_directory(
    train_data_dir,
    batch_size = BATCH_SIZE,
    image_size = IMG_SIZE,
    subset = 'training',
    validation_split = 0.2,
    seed = 42
)

val_data = tf.keras.utils.image_dataset_from_directory(
    val_data_dir,
    batch_size = BATCH_SIZE,
    image_size = IMG_SIZE,
    subset = 'validation',
    validation_split = 0.2,
    seed = 42
)

test_data = tf.keras.utils.image_dataset_from_directory(
    test_data_dir,
    batch_size = BATCH_SIZE,
    image_size = IMG_SIZE,
)

Found 5466 files belonging to 5 classes.
Using 4373 files for training.
Found 1819 files belonging to 5 classes.
Using 363 files for validation.
Found 1829 files belonging to 5 classes.


In [6]:
class_names = train_data.class_names
class_names

['H1', 'H2', 'H3', 'H5', 'H6']